# Finished Full Code

## Downloads

In [ ]:
# @title Install dependencies
!pip install nilearn --quiet

     |████████████████████████████████| 9.6 MB 4.5 MB/s 


In [ ]:
# @title Download data file
import os, requests

fname = "hcp_task.tgz"
url = "https://osf.io/2y3fw/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp_task"

# importing the "tarfile" module
import tarfile

# open file
with tarfile.open(fname) as tfile:
  # extracting file
  tfile.extractall('.')

subjects = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')

NameError: ignored

In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

## Initialization

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nilearn
import sklearn

In [ ]:
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'
# TIP: look inside the data folders for more fine-graned conditions

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['ment','rnd']}
}

## Helper functions

In [ ]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


def load_evs(subject, experiment, run):
  """Load EVs (explanatory variables) data for one task experiment.

  Args:
    subject (str): subject ID to load
    experiment (str) : Name of experiment
    run (int): 0 or 1

  Returns
    evs (list of lists): A list of frames associated with each condition

  """
  frames_list = []
  task_key = f'tfMRI_{experiment}_{RUNS[run]}'
  for cond in EXPERIMENTS[experiment]['cond']:
    ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)
    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)
    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
    frames[-1] = [i for i in frames[-1] if i < 316]
    frames_list.append(frames)

  return frames_list

In [ ]:
def block_glm_binaries(data, evs, experiment, cond):
  
  # Find indices/time points of conditions (e.g., math or story)
  idx = EXPERIMENTS[experiment]['cond'].index(cond) # get the index of the condition
  blocks = [evs[idx][i] for i in range(len(evs[idx]))] # concatenate index for condition
  
  # Create binarized vector of conditions
  bins = np.zeros((data.shape[1], len(evs[idx])))
  for b in range(0, len(blocks)):
     bin = np.zeros(data.shape[1]) # instantiate binary vector
     bin[blocks[b]] = 1 # make identified indices one 
     bins[:,b] = bin
  return bins

In [ ]:
def block_tdm_binary(data, evs, experiment, cond):
  
  # Find indices/time points of conditions (e.g., math or story)
  idx = EXPERIMENTS[experiment]['cond'].index(cond) # get the index of the condition
  block = np.concatenate([evs[idx][i] for i in range(len(evs[idx]))], axis=-1) # concatenate index for condition
  
  # Create binarized vector of conditions
  bin = np.zeros(data.shape[1]) # instantiate binary vector
  bin[block] = 1 # make identified indices one 
  return bin

In [ ]:
def glm(Y, X, C=None, mask=None):
  """
  Run a general linear model

  Args:
    Y (2d array) : time-by-space data matrix
    X (2d array) : time-by-regressors design matrix
    C (2d array) : contrasts-by-regressor contrrast matrix [default=Identity]
    mask (1d array) : spatial mask wherre GLM is run

  Returns:
    contrast maps
    t-stats
  """
  if C is None:
    C = np.identity(X.shape[1])
  if mask is None:
    mask = np.ones(Y.shape[1])

  # initialise matrices
  beta = np.zeros((X.shape[1], Y.shape[1]))
  cope = np.zeros((C.shape[0], Y.shape[1]))
  varbeta = np.zeros_like(beta)
  tstat = np.zeros_like(beta)

  # solve glm
  beta[:, mask > 0] = np.linalg.pinv(X) @ Y[:, mask > 0]
  # apply contrasts
  cope[:, mask > 0] = np.dot(C, beta[:, mask > 0])

  # calculate uncertainty (varcope)
  r = Y - X@beta
  dof = X.shape[0] - np.linalg.matrix_rank(X)
  sig2 = np.sum(r**2, axis=0) / dof
  varcope = np.outer(C @ np.diag(np.linalg.inv(X.T @ X)) @ C.T, sig2)
  # calculate t-stats
  #tstat[:, mask] = cope[:, mask] / np.sqrt(varcope[:, mask])

  return cope #, tstat

In [ ]:
def gen_glm_design_matrix(data, evs, experiment, conds):
  
    design_matrix = np.ones((data.shape[1], len(evs[0]) + len(evs[1]) + 1))

    # print(design_matrix.shape)

    idx = 1

    for i in range(0,len(evs[0])+len(evs[1])):
        if i < len(evs[0]):
            design_matrix[:, idx] = block_glm_binaries(data, evs, experiment, conds[0])[:,i]
        else:
            design_matrix[:, idx] = block_glm_binaries(data, evs, experiment, conds[1])[:,i-len(evs[0])]
        idx += 1
    return design_matrix

In [ ]:
def gen_tdm_design_matrix(data, evs, experiment, conds):
  design_matrix = np.ones((data.shape[1], len(conds)))
  idx = 0
  for cond in conds:
    design_matrix[:, idx] = block_tdm_binary(data, evs, experiment, cond) 
    #np.convole(block_binary(data, evs, experiment, cond), HRF, 'full')[:data.shape[1]]    
    #np.convolve(story_bin, HRF, 'full')[:data.shape[1]]
    idx += 1
  return design_matrix

In [ ]:
def gen_target_vec(evs):
  target = np.zeros(len(evs[0])+len(evs[1]))
  target[:len(evs[0])] = 1
  return target

In [ ]:
def median_duration(evs1, evs2):
  dur = []
  for i in range(len(evs1)):
    for j in range(len(evs1[i])):
        dur.append(len(evs1[i][j]))
  for i in range(len(evs2)):
    for j in range(len(evs2[i])):
        dur.append(len(evs2[i][j]))
  med_dur = np.median(np.array(dur))
  return np.round(med_dur)

In [ ]:
def reduce_evs(window, evs, target, bold):
  idx = 0
  final_targ = []
  final_evs = [[],[]]
  for i in range(len(evs)):
    for j in range(len(evs[i])):
      onset = evs[i][j][0]
      if (onset + window) < bold.shape[1]:
        final_targ.append(target[idx])
        final_evs[i].append(evs[i][j])
      idx += 1
  final_targ = np.array(final_targ)
  return final_targ, final_evs

In [ ]:
def gen_window_vals(window, evs, pred):
  val = []
  for i in range(len(evs)):
    for j in range(len(evs[i])):
      onset = evs[i][j][0]
      val.append(np.hstack((pred[onset:(onset+int(window)), 0], pred[onset:(onset+int(window)), 1])))
  val = np.array(val)
  return val

In [ ]:
def lateralize(data):
    hemis = region_info['hemi']
    l_i = hemis.index('Left')
    
    left = data[l_i:,:]
    right = data[:l_i,:]

    return left, right

## Run training and testing

In [ ]:
my_exp = 'LANGUAGE'

window = []

for subj in subjects:
    # Extract timeseries
    bold_train, bold_test = load_single_timeseries(subj, my_exp, 0, remove_mean = False), load_single_timeseries(subj, my_exp, 1, remove_mean = False)  

    # Extract events
    evs_train, evs_test = load_evs(subj, my_exp, 0), load_evs(subj, my_exp, 1)

    # Generate Logistic Windows
    window.append(median_duration(evs_train, evs_test))
window_med = np.median(np.array(window))

logit_window = np.round(window_med)

In [ ]:
import sklearn.linear_model



#  Runs training and testing for the first n subjects (n_subjects), and returns their accuracies
def run(n_subjects):
    my_exp = 'LANGUAGE'
    l_acc_glm = []
    r_acc_glm = []
    l_acc_tdm = []
    r_acc_tdm = [] 

    # Stats dictionary
    subject_stats = {}
    for subj in subjects[:n_subjects]:
        subject_stats[subj] = {}
        subject_stats[subj]['glm'] = {}
        subject_stats[subj]['tdm'] = {}


        # Extract timeseries
        bold_train, bold_test = load_single_timeseries(subj, my_exp, 0), load_single_timeseries(subj, my_exp, 1)  

        # Lateralize bold data
        l_bold_train, r_bold_train = lateralize(bold_train)
        l_bold_test, r_bold_test = lateralize(bold_test)

        # Extract events
        evs_train, evs_test = load_evs(subj, my_exp, 0), load_evs(subj, my_exp, 1)
        
        # Create targets
        target_train = gen_target_vec(evs_train)
        target_test = gen_target_vec(evs_test)

        subject_stats[subj]['target_train'] = target_train.tolist()

        # Reduce EVS
        target_train, evs_train = reduce_evs(logit_window, evs_train, target_train, bold_train)
        target_test, evs_test = reduce_evs(logit_window, evs_test, target_test, bold_test)

        ## GLM Analysis
        
        # Create design matrices
        l_design_glm_train = gen_glm_design_matrix(l_bold_train, evs_train, my_exp, ['math', 'story'])
        r_design_glm_train = gen_glm_design_matrix(r_bold_train, evs_train, my_exp, ['math', 'story'])
        l_design_glm_test = gen_glm_design_matrix(l_bold_test, evs_test, my_exp, ['math', 'story'])
        r_design_glm_test = gen_glm_design_matrix(r_bold_test, evs_test, my_exp, ['math', 'story'])

        subject_stats[subj]['glm']['l_DMat'] = l_design_glm_train.tolist()
        subject_stats[subj]['glm']['r_DMat'] = r_design_glm_train.tolist()

        # Generate beta values
        l_beta_train =  glm(l_bold_train.T, l_design_glm_train)
        r_beta_train =  glm(r_bold_train.T, r_design_glm_train)
        l_beta_test = glm(l_bold_train.T, l_design_glm_test)
        r_beta_test = glm(r_bold_train.T, l_design_glm_test)

        subject_stats[subj]['glm']['l_beta'] = l_beta_train.tolist()
        subject_stats[subj]['glm']['r_beta'] = r_beta_train.tolist()

        # Fit logistic regression with train
        l_glm_log = sklearn.linear_model.LogisticRegressionCV(random_state=2022)
        l_glm_log.fit(l_beta_train[1:, ], target_train)

        r_glm_log = sklearn.linear_model.LogisticRegressionCV(random_state=2022)
        r_glm_log.fit(r_beta_train[1:, ], target_train)
        
        # Generate predictions of model on test data
        l_glm_pred = l_glm_log.predict(l_beta_test[1:, ])
        r_glm_pred = r_glm_log.predict(r_beta_test[1:, ])

        # Store accuracy on test data
        l_acc = np.mean(l_glm_pred==target_test)
        l_acc_glm.append(l_acc)
        r_acc = np.mean(r_glm_pred==target_test)
        r_acc_glm.append(r_acc)

        subject_stats[subj]['glm']['l_acc'] = l_acc 
        subject_stats[subj]['glm']['r_acc'] = r_acc

        ## Time Decoding Analysis

        # Create design matrices
        l_design_tdm_train = gen_tdm_design_matrix(l_bold_train, evs_train, my_exp, ['math', 'story'])
        r_design_tdm_train = gen_tdm_design_matrix(r_bold_train, evs_train, my_exp, ['math', 'story'])
        l_design_tdm_test = gen_tdm_design_matrix(l_bold_test, evs_test, my_exp, ['math', 'story'])
        r_design_tdm_test = gen_tdm_design_matrix(r_bold_test, evs_test, my_exp, ['math', 'story'])

        subject_stats[subj]['tdm']['l_DMat'] = l_design_tdm_train.tolist()
        subject_stats[subj]['tdm']['r_DMat'] = r_design_tdm_train.tolist()

        # Fit ridge regression
        
        # Define model
        n_alpha = 5 # number of lambdas to test
        alphas = np.logspace(- n_alpha / 2, n_alpha - (n_alpha / 2), num=n_alpha)
        l_ridge = sklearn.linear_model.RidgeCV(alphas=alphas)
        r_ridge = sklearn.linear_model.RidgeCV(alphas=alphas)

        # Fit model to train data
        l_ridge.fit(l_bold_train.T, l_design_tdm_train)
        r_ridge.fit(r_bold_train.T, r_design_tdm_train)

        subject_stats[subj]['tdm']['l_beta'] = l_ridge.coef_.tolist()
        subject_stats[subj]['tdm']['r_beta'] = r_ridge.coef_.tolist()

        # Predict stimulus train given fmri
        l_prediction_train = l_ridge.predict(l_bold_train.T)
        r_prediction_train = r_ridge.predict(r_bold_train.T)

        # Predict stimulus test given fmri
        l_prediction_test = l_ridge.predict(l_bold_test.T)
        r_prediction_test = r_ridge.predict(r_bold_test.T)

        # Create windows for each event
        l_window_vals_train = gen_window_vals(logit_window, evs_train, l_prediction_train)
        r_window_vals_train = gen_window_vals(logit_window, evs_train, r_prediction_train)
        l_window_vals_test = gen_window_vals(logit_window, evs_test, l_prediction_test)
        r_window_vals_test = gen_window_vals(logit_window, evs_test, r_prediction_test)

        # Fit logistic regression
        
        # Define Model
        l_tdm_log = sklearn.linear_model.LogisticRegressionCV(random_state=2022)
        r_tdm_log = sklearn.linear_model.LogisticRegressionCV(random_state=2022)
        
        # Fit Model
        l_tdm_log.fit(l_window_vals_train, target_train)
        r_tdm_log.fit(r_window_vals_train, target_train)



        # Generate predictions of model on test data
        l_tdm_pred = l_tdm_log.predict(l_window_vals_test)
        r_tdm_pred = r_tdm_log.predict(r_window_vals_test)

        # Store accuracy on test data
        l_acc = np.mean(l_tdm_pred==target_test)
        l_acc_tdm.append(l_acc)
        r_acc = np.mean(r_tdm_pred==target_test)
        r_acc_tdm.append(r_acc)

        subject_stats[subj]['tdm']['l_acc'] = l_acc
        subject_stats[subj]['tdm']['r_acc'] = r_acc

    return l_acc_glm, r_acc_glm, l_acc_tdm, r_acc_tdm, subject_stats

a,b,c,d,_ = run(1)

_['100307']['target_train']




In [ ]:
np.array(_['100307']['glm']['l_beta']).shape

In [ ]:
bold_train, bold_test = load_single_timeseries(subj, my_exp, 0), load_single_timeseries(subj, my_exp, 1)  

## Visualization

### What to do

*   3D Brain map contrast of betas (language vs math)
*   Compare accuracies between the two methods (GLM vs TDM)
*   Compare accuracies between the two hemispheres on the language task (Left vs Right)



## Example run data

In [ ]:
l_acc_glm, r_acc_glm, l_acc_tdm, r_acc_tdm, subject_stats = run(100)

In [ ]:
subject_stats['100307']['target_train']

## Design Matrix Plots 

In [ ]:
plt.figure()
plt.plot(np.array(subject_stats['100307']['tdm']['l_DMat']))
plt.legend(['math', 'story'], loc = 'upper right')


## Accuracy Graphs

In [ ]:
fig, ax = plt.subplots()

# Left hemi
ax.set_title('Left Hemi - Accuracies')
plt.plot(l_acc_glm, label='glm')
plt.plot(l_acc_tdm, label='tdm')
plt.legend()
plt.show()

# Right hemi
ax.set_title('Right Hemi - Accuracies')
plt.plot(r_acc_glm, label='glm')
plt.plot(r_acc_tdm, label='tdm')
plt.legend()
plt.show()

In [ ]:
# Dataset 

#l_acc_glm, r_acc_glm, l_acc_tdm, r_acc_tdm
data_GLM_L = l_acc_glm
data_TDM_L = l_acc_tdm
data_GLM_R = r_acc_glm
data_TDM_R = r_acc_tdm

data = [data_GLM_L, data_TDM_L, data_GLM_R, data_TDM_R]

fig = plt.figure(figsize =(8, 4))
 
# Creating axes instance
ax = plt.gca()

#labeling x axis
ax.set_xticklabels(['GLM_L', 'TDM_L',
					'GLM_R', 'TDM_R'])
 
# Creating plot
bp = ax.boxplot(data)
plt.title('Accuracy Scores')
plt.ylabel(("Accuracy"))

#reference line
left, right = plt.xlim()
plt.hlines(0.5, xmin=left, xmax=right, color='r', linestyles='--')

# show plot
plt.show()

## Accuracy Stats

In [ ]:
from scipy import stats
l_tStat, l_pValue = stats.ttest_rel(l_acc_glm, l_acc_tdm)

print(l_tStat, l_pValue)

r_tStat, r_pValue = stats.ttest_rel(r_acc_glm, r_acc_tdm)

print(r_tStat, r_pValue)

lr_tStat, lr_pValue = stats.ttest_rel(l_acc_tdm, r_acc_tdm)

print(lr_tStat, lr_pValue)


In [ ]:
med_l_glm = np.median(np.array(l_acc_glm))
med_r_glm = np.median(np.array(r_acc_glm))
print(med_l_glm, med_r_glm)

med_l_tdm = np.median(np.array(l_acc_tdm))
med_r_tdm = np.median(np.array(r_acc_tdm))
print(med_l_tdm, med_r_tdm)

## Brain map

In [ ]:
# @title NMA provides an atlas. Run this cell to download it
import os, requests

# NMA provides an atlas
fname = f"{HCP_DIR}/atlas.npz"
url = "https://osf.io/j5kuc/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

with np.load(fname) as dobj:
  atlas = dict(**dobj)

In [ ]:
def get_mean_betas(method='glm',hemi='l', subject_stats={}):
    math_mean_betas = []
    lang_mean_betas = []

    for subj in subjects:
        subj_targets = np.array(subject_stats[subj]['target_train'])
        subj_betas = subject_stats[subj][method][hemi+'_beta']
        num_betas = np.array(subj_betas)


        if method == 'glm':
            math_idxs = np.where(subj_targets == 1)[0]
            math_betas = num_betas[math_idxs, :]

            lang_idxs = np.where(subj_targets == 0)[0]
            lang_betas = num_betas[lang_idxs, :]

            mean_math_betas = np.mean(math_betas, axis = 0)      
            mean_lang_betas = np.mean(lang_betas, axis = 0)
            
            math_mean_betas.append(mean_math_betas)
            lang_mean_betas.append(mean_lang_betas)

        else:
            math_betas = num_betas[0, :]
            lang_betas = num_betas[1, :]

            math_mean_betas.append(math_betas)
            lang_mean_betas.append(lang_betas)


    math_avg_mean_betas = np.mean(np.array(math_mean_betas), axis = 0)
    lang_avg_mean_betas = np.mean(np.array(lang_mean_betas), axis = 0)

    return math_avg_mean_betas, lang_avg_mean_betas

In [ ]:
def get_mean_betas(method='glm',hemi='l', subject_stats={}):
    math_mean_betas = []
    lang_mean_betas = []

    for subj in subjects:
        subj_targets = np.array(subject_stats[subj]['target_train'])
        subj_betas = subject_stats[subj][method][hemi+'_beta']
        num_betas = np.array(subj_betas)

        math_idxs = np.where(subj_targets == 1)[0]
        math_betas = num_betas[math_idxs, :]
        mean_math_betas = np.mean(math_betas, axis = 0)
        
        lang_idxs = np.where(subj_targets == 0)[0]
        lang_betas = num_betas[lang_idxs, :]
        mean_lang_betas = np.mean(lang_betas, axis = 0)

        math_mean_betas.append(mean_math_betas)
        lang_mean_betas.append(mean_lang_betas)

    math_avg_mean_betas = np.mean(np.array(math_mean_betas), axis = 0)
    lang_avg_mean_betas = np.mean(np.array(lang_mean_betas), axis = 0)

    return math_avg_mean_betas, lang_avg_mean_betas

### GLM Brain Plot

In [ ]:
l_math_avg, l_lang_avg = get_mean_betas('glm','l',subject_stats)
r_math_avg, r_lang_avg = get_mean_betas('glm','r',subject_stats)

math_avg = np.concatenate([r_math_avg, l_math_avg])
lang_avg = np.concatenate([r_lang_avg, l_lang_avg])

In [ ]:
group_contrast = 0

glm_contrast =  lang_avg - math_avg

# This uses the nilearn package
from nilearn import plotting, datasets

# Try both hemispheres (L->R and left->right)
fsaverage = datasets.fetch_surf_fsaverage()
surf_contrast = glm_contrast[atlas["labels_L"]]
plotting.view_surf(fsaverage['infl_left'],
                   surf_contrast,
                   vmax=100)

In [ ]:
# Try both hemispheres (L->R and left->right)
fsaverage = datasets.fetch_surf_fsaverage()
surf_contrast = glm_contrast[atlas["labels_R"]]
plotting.view_surf(fsaverage['infl_right'],
                   surf_contrast,
                   vmax=100)

### TDM Brain Plot

In [ ]:
l_math_avg, l_lang_avg = get_mean_betas('tdm','l',subject_stats)
r_math_avg, r_lang_avg = get_mean_betas('tdm','r',subject_stats)

tdm_math_avg = np.concatenate([r_math_avg, l_math_avg])
tdm_lang_avg = np.concatenate([r_lang_avg, l_lang_avg])

In [ ]:
group_contrast = 0

tdm_contrast =  tdm_lang_avg - tdm_math_avg

# This uses the nilearn package
from nilearn import plotting, datasets

# Try both hemispheres (L->R and left->right)
fsaverage = datasets.fetch_surf_fsaverage()
surf_contrast = tdm_contrast[atlas["labels_L"]]
plotting.view_surf(fsaverage['infl_left'],
                   surf_contrast,
                   vmax=0.0015)

In [ ]:
# Try both hemispheres (L->R and left->right)
fsaverage = datasets.fetch_surf_fsaverage()
surf_contrast = tdm_contrast[atlas["labels_R"]]
plotting.view_surf(fsaverage['infl_right'],
                   surf_contrast,
                   vmax=0.0015)